# Unistore Latency Spikes Investigation

In [ ]:
SELECT
    TIME,
    MACHINE,
    v:"P99"::float as latency_p99
FROM
    SNOWHOUSE_IMPORT.PROD1.FDB_SERVER_LOGS_OLTP_V
WHERE
    Type = 'NormalReadLatencyMetrics'
    AND log_group = 'prod1_autoprovision_88_100_fdbserver'
    AND TIME BETWEEN '2025-10-31 00:00:00'::timestamp_ntz
    AND '2025-11-06 00:00:00'::timestamp_ntz;

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

df = latency.to_pandas()

# Display basic info about the dataset
print(f"Dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

# Show first few rows
print("\nFirst 5 rows:")
print(df.head())

# Plot latency over time as scatter plot (points without connecting lines)
plt.figure(figsize=(12, 6))

# Create boolean mask for high latency
high_latency_mask = df['LATENCY_P99'] > 0.5

# Plot normal latency in blue
normal_data = df[~high_latency_mask]
if not normal_data.empty:
    plt.scatter(normal_data['TIME'], normal_data['LATENCY_P99'], 
               color='blue', alpha=0.7, label='Normal (<= 0.5s)', s=20)

# Plot high latency in red
high_data = df[high_latency_mask]
if not high_data.empty:
    plt.scatter(high_data['TIME'], high_data['LATENCY_P99'], 
               color='red', alpha=0.7, label='High (> 0.5s)', s=20)

plt.xlabel('Time')
plt.ylabel('Latency P99 (seconds)')
plt.title('prod1_autoprovision_88_100 P99 Latency Over Time (1 week)')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

# Also create a Streamlit version
st.subheader("prod1_autoprovision_88_100 P99 Latency 1 week")
st.line_chart(df.set_index('TIME')['LATENCY_P99'])


In [ ]:
WITH high_latency_events AS (
    SELECT
        TIME,
        MACHINE,
        v:"P99"::float as latency_p99
    FROM
        SNOWHOUSE_IMPORT.PROD1.FDB_SERVER_LOGS_OLTP_V
    WHERE
        Type = 'NormalReadLatencyMetrics'
        AND v:"P99"::float > 0.5
        AND log_group = 'prod1_autoprovision_88_100_fdbserver'
        AND TIME BETWEEN '2025-10-31 00:00:00'::timestamp_ntz
        AND '2025-11-06 00:00:00'::timestamp_ntz
),
redwood_metrics AS (
    SELECT
        TIME,
        MACHINE,
        v:OpGetRange::float as op_get_range,
        COALESCE(v:OpTryNext::float, 0) + 
        COALESCE(v:OpNext::float, 0) + 
        COALESCE(v:OpNextBatch::float, 0) as op_nexts
    FROM
        SNOWHOUSE_IMPORT.PROD1.FDB_SERVER_LOGS_OLTP_V
    WHERE
        Type = 'RedwoodMetrics'
        AND log_group = 'prod1_autoprovision_88_100_fdbserver'
        AND TIME BETWEEN '2025-10-31 00:00:00'::timestamp_ntz
        AND '2025-11-06 00:00:00'::timestamp_ntz
),
storage_metrics AS (
    SELECT
        TIME,
        MACHINE,
        split_part(v:ClearRangeMutations, ' ', 1)::float as clear_range_mutations,
        split_part(v:GetRangeQueries, ' ', 1)::float as ss_get_range
    FROM
        SNOWHOUSE_IMPORT.PROD1.FDB_SERVER_LOGS_OLTP_V
    WHERE
        Type = 'StorageMetrics'
        AND log_group = 'prod1_autoprovision_88_100_fdbserver'
        AND TIME BETWEEN '2025-10-31 00:00:00'::timestamp_ntz
        AND '2025-11-06 00:00:00'::timestamp_ntz
        AND v:ClearRangeMutations IS NOT NULL
),
read_version_wait_metrics AS (
    SELECT
        TIME,
        MACHINE,
        v:"P99"::float as read_version_wait_p99
    FROM
        SNOWHOUSE_IMPORT.PROD1.FDB_SERVER_LOGS_OLTP_V
    WHERE
        Type = 'ReadVersionWaitMetrics'
        AND log_group = 'prod1_autoprovision_88_100_fdbserver'
        AND TIME BETWEEN '2025-10-31 00:00:00'::timestamp_ntz
        AND '2025-11-06 00:00:00'::timestamp_ntz
        AND v:"P99" IS NOT NULL
),
read_queue_wait_metrics AS (
    SELECT
        TIME,
        MACHINE,
        v:"P99"::float as read_queue_wait_p99
    FROM
        SNOWHOUSE_IMPORT.PROD1.FDB_SERVER_LOGS_OLTP_V
    WHERE
        Type = 'ReadQueueWaitMetrics'
        AND log_group = 'prod1_autoprovision_88_100_fdbserver'
        AND TIME BETWEEN '2025-10-31 00:00:00'::timestamp_ntz
        AND '2025-11-06 00:00:00'::timestamp_ntz
        AND v:"P99" IS NOT NULL
)
SELECT
    h.TIME as high_latency_time,
    h.MACHINE,
    h.latency_p99,
    SUM(r.op_get_range) as total_op_get_range_1min_before,
    SUM(r.op_nexts) as total_op_nexts_1min_before,
    SUM(s.clear_range_mutations) as total_clear_range_mutations_1min_before,
    SUM(s.ss_get_range) as total_ss_get_range_1min_before,
    MAX(rv.read_version_wait_p99) as max_read_version_wait_p99_1min_before,
    MAX(rq.read_queue_wait_p99) as max_read_queue_wait_p99_1min_before
FROM
    high_latency_events h
    LEFT JOIN redwood_metrics r ON h.MACHINE = r.MACHINE
        AND r.TIME BETWEEN DATEADD('minute', -1, h.TIME)
        AND DATEADD('second', 5, h.TIME)
    LEFT JOIN storage_metrics s ON h.MACHINE = s.MACHINE
        AND s.TIME BETWEEN DATEADD('minute', -1, h.TIME)
        AND DATEADD('second', 5, h.TIME)
    LEFT JOIN read_version_wait_metrics rv ON h.MACHINE = rv.MACHINE
        AND rv.TIME BETWEEN DATEADD('minute', -1, h.TIME)
        AND DATEADD('minute', 1, h.TIME)
    LEFT JOIN read_queue_wait_metrics rq ON h.MACHINE = rq.MACHINE
        AND rq.TIME BETWEEN DATEADD('minute', -1, h.TIME)
        AND DATEADD('minute', 1, h.TIME)
GROUP BY
    h.TIME,
    h.MACHINE,
    h.latency_p99
-- HAVING
    -- SUM(r.op_get_range) < 100000
--     AND SUM(r.op_nexts) < 10000
--     AND SUM(s.ss_get_range) > 100
ORDER BY
    h.TIME DESC;

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Get data from latency_get_range SQL cell
df_get_range = latency_get_range.to_pandas()

# Create boolean mask for high operation count (increased to 100K)
high_op_mask = df_get_range['TOTAL_OP_GET_RANGE_1MIN_BEFORE'] > 100000

# Create figure with dual y-axes
fig, ax1 = plt.subplots(figsize=(15, 8))

# Plot latency on primary y-axis with conditional coloring
ax1.set_xlabel('Time')
ax1.set_ylabel('Latency P99 (seconds)', color='red')

# Plot points based on operation count threshold (smaller dots)
match_data = df_get_range[high_op_mask]
not_match_data = df_get_range[~high_op_mask]

if not match_data.empty:
    ax1.scatter(match_data['HIGH_LATENCY_TIME'], match_data['LATENCY_P99'], 
               color='red', alpha=0.8, s=20, label='Match (>100K ops)', marker='o')

if not not_match_data.empty:
    ax1.scatter(not_match_data['HIGH_LATENCY_TIME'], not_match_data['LATENCY_P99'], 
               color='orange', alpha=0.8, s=20, label='Not Match (≤100K ops)', marker='x')

ax1.tick_params(axis='y', labelcolor='red')
ax1.tick_params(axis='x', rotation=45)

# Create second y-axis for operation count
ax2 = ax1.twinx()
ax2.set_ylabel('Total Op Get Range (1min before)', color='blue')

# Plot operation counts with same conditional coloring (smaller dots)
if not match_data.empty:
    ax2.scatter(match_data['HIGH_LATENCY_TIME'], match_data['TOTAL_OP_GET_RANGE_1MIN_BEFORE'], 
               color='darkblue', alpha=0.8, s=20, marker='o')

if not not_match_data.empty:
    ax2.scatter(not_match_data['HIGH_LATENCY_TIME'], not_match_data['TOTAL_OP_GET_RANGE_1MIN_BEFORE'], 
               color='lightblue', alpha=0.8, s=20, marker='x')

ax2.tick_params(axis='y', labelcolor='blue')

# Add horizontal line at 100,000 threshold
ax2.axhline(y=100000, color='gray', linestyle='--', alpha=0.7, label='100K threshold')

# Add title and grid
plt.title('Latency Spikes vs Operation Load (Match: >100K ops, Not Match: ≤100K ops)')
ax1.grid(True, alpha=0.3)

# Add legends
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

plt.tight_layout()
plt.show()

# Calculate summary with percentages
match_count = len(match_data)
not_match_count = len(not_match_data)
total_count = match_count + not_match_count

match_percentage = (match_count / total_count * 100) if total_count > 0 else 0
not_match_percentage = (not_match_count / total_count * 100) if total_count > 0 else 0

# Print summary with percentages
print(f"Match (>100K ops): {match_count} events ({match_percentage:.1f}%)")
print(f"Not Match (≤100K ops): {not_match_count} events ({not_match_percentage:.1f}%)")
print(f"Total events: {total_count}")

# Also create Streamlit version
st.subheader("Latency vs Operation Load Analysis")
st.write(f"**Match (>100K ops):** {match_count} events ({match_percentage:.1f}%) | **Not Match (≤100K ops):** {not_match_count} events ({not_match_percentage:.1f}%)")
st.write(f"**Total events:** {total_count}")
st.write("Red circles = Match, Orange X = Not Match")

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Get data from latency_get_range SQL cell and filter for TOTAL_OP_GET_RANGE_1MIN_BEFORE < 100000
df_nexts = latency_get_range.to_pandas()
df_filtered = df_nexts[df_nexts['TOTAL_OP_GET_RANGE_1MIN_BEFORE'] < 100000].copy()

# Create boolean mask for high op_nexts count (match when > 100000)
high_nexts_mask = df_filtered['TOTAL_OP_NEXTS_1MIN_BEFORE'] > 100000

# Create figure with dual y-axes
fig, ax1 = plt.subplots(figsize=(15, 8))

# Plot latency on primary y-axis with conditional coloring
ax1.set_xlabel('Time')
ax1.set_ylabel('Latency P99 (seconds)', color='red')

# Plot points based on op_nexts threshold
match_data = df_filtered[high_nexts_mask]
not_match_data = df_filtered[~high_nexts_mask]

if not match_data.empty:
    ax1.scatter(match_data['HIGH_LATENCY_TIME'], match_data['LATENCY_P99'], 
               color='red', alpha=0.8, s=20, label='Match (>100K nexts)', marker='o')

if not not_match_data.empty:
    ax1.scatter(not_match_data['HIGH_LATENCY_TIME'], not_match_data['LATENCY_P99'], 
               color='orange', alpha=0.8, s=20, label='Not Match (≤100K nexts)', marker='x')

ax1.tick_params(axis='y', labelcolor='red')
ax1.tick_params(axis='x', rotation=45)

# Create second y-axis for op_nexts count
ax2 = ax1.twinx()
ax2.set_ylabel('Total Op Nexts (1min before)', color='blue')

# Plot op_nexts counts with same conditional coloring
if not match_data.empty:
    ax2.scatter(match_data['HIGH_LATENCY_TIME'], match_data['TOTAL_OP_NEXTS_1MIN_BEFORE'], 
               color='darkblue', alpha=0.8, s=20, marker='o')

if not not_match_data.empty:
    ax2.scatter(not_match_data['HIGH_LATENCY_TIME'], not_match_data['TOTAL_OP_NEXTS_1MIN_BEFORE'], 
               color='lightblue', alpha=0.8, s=20, marker='x')

ax2.tick_params(axis='y', labelcolor='blue')

# Add horizontal line at 100,000 threshold
ax2.axhline(y=100000, color='gray', linestyle='--', alpha=0.7, label='100K nexts threshold')

# Add title and grid
plt.title('Latency Spikes vs Op Nexts (Filtered: Op Get Range < 100K)')
ax1.grid(True, alpha=0.3)

# Add legends
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

plt.tight_layout()
plt.show()

# Calculate summary with percentages
match_count = len(match_data)
not_match_count = len(not_match_data)
total_count = match_count + not_match_count

match_percentage = (match_count / total_count * 100) if total_count > 0 else 0
not_match_percentage = (not_match_count / total_count * 100) if total_count > 0 else 0

# Print summary with percentages
print(f"Filtered dataset (Op Get Range < 100K): {total_count} events")
print(f"Match (>100K nexts): {match_count} events ({match_percentage:.1f}%)")
print(f"Not Match (≤100K nexts): {not_match_count} events ({not_match_percentage:.1f}%)")

# Also create Streamlit version
st.subheader("Iterator Next Operations Analysis (Filtered)")
st.write(f"**Filtered dataset (Op Get Range < 100K):** {total_count} events")
st.write(f"**Match (>100K nexts):** {match_count} events ({match_percentage:.1f}%) | **Not Match (≤100K nexts):** {not_match_count} events ({not_match_percentage:.1f}%)")
st.write("Red circles = Match, Orange X = Not Match")

In [ ]:
-- Welcome to Snowflake Notebooks!
-- Try out a SQL cell to generate some data.
select
  time_slice(time, 300, 'second') as TIMEBUCKET,
  machine,
  sum(
      iff(
          Type = 'RedwoodMetrics',
          v:OpGetRange::float,
          null
      )
  ) as query_num,
  max(
      iff(
          Type = 'NormalReadLatencyMetrics',
          v:"P99"::float,
          null
      )
  ) as latency_p99,
  sum(iff(Type = 'StorageMetrics', split_part(v:FetchRangeReadBytes, ' ', 1)::float, null)) as fetch_bytes,
  sum(iff(Type = 'StorageMetrics', split_part(v:FetchRangeReadCount, ' ', 1)::float, null)) as fetch_count,
  sum(iff(Type = 'StorageMetrics', split_part(v:RowsQueried, ' ', 1)::float, null)) as rows_read,
  iff(
    sum(
      iff(
        Type = 'RedwoodMetrics',
        v:OpGetRange::float,
        null
      )
    ) = 0,
    null,
    sum(iff(Type = 'StorageMetrics', split_part(v:FetchRangeReadBytes, ' ', 1)::float, null)) /
    sum(
      iff(
        Type = 'RedwoodMetrics',
        v:OpGetRange::float,
        null
      )
    )
  ) as bytes_per_query,
    iff(
    sum(
      iff(
        Type = 'RedwoodMetrics',
        v:OpGetRange::float,
        null
      )
    ) = 0,
    null,
    sum(iff(Type = 'StorageMetrics', split_part(v:RowsQueried, ' ', 1)::float, null)) /
    sum(
      iff(
        Type = 'RedwoodMetrics',
        v:OpGetRange::float,
        null
      )
    )
  ) as rows_per_query
from
  SNOWHOUSE_IMPORT.PROD1.FDB_SERVER_LOGS_OLTP_V
where
  Type in ('RedwoodMetrics', 'NormalReadLatencyMetrics', 'StorageMetrics')
  and log_group = 'prod1_autoprovision_88_100_fdbserver'
  and TIMEBUCKET between '2025-11-01 00:00:00'::timestamp_ntz
  and '2025-11-06 00:00:00'::timestamp_ntz
group by
  TIMEBUCKET, machine
having
  latency_p99 > 1
order by latency_p99 desc;

In [ ]:
WITH high_latency_events AS (
    SELECT
        TIME,
        MACHINE,
        v:"P99"::float as latency_p99
    FROM
        SNOWHOUSE_IMPORT.PROD1.FDB_SERVER_LOGS_OLTP_V
    WHERE
        Type = 'NormalReadLatencyMetrics'
        AND v:"P99"::float > 0.1
        AND log_group = 'prod1_autoprovision_88_100_fdbserver'
        AND TIME BETWEEN '2025-10-31 00:00:00'::timestamp_ntz
        AND '2025-11-06 00:00:00'::timestamp_ntz
),
redwood_metrics AS (
    SELECT
        TIME,
        MACHINE,
        v:OpGetRange::float as op_get_range,
        COALESCE(v:OpTryNext::float, 0) + 
        COALESCE(v:OpNext::float, 0) + 
        COALESCE(v:OpNextBatch::float, 0) as op_nexts
    FROM
        SNOWHOUSE_IMPORT.PROD1.FDB_SERVER_LOGS_OLTP_V
    WHERE
        Type = 'RedwoodMetrics'
        AND log_group = 'prod1_autoprovision_88_100_fdbserver'
        AND TIME BETWEEN '2025-10-31 00:00:00'::timestamp_ntz
        AND '2025-11-06 00:00:00'::timestamp_ntz
),
storage_metrics AS (
    SELECT
        TIME,
        MACHINE,
        split_part(v:ClearRangeMutations, ' ', 1)::float as clear_range_mutations,
        split_part(v:GetRangeQueries, ' ', 1)::float as ss_get_range
    FROM
        SNOWHOUSE_IMPORT.PROD1.FDB_SERVER_LOGS_OLTP_V
    WHERE
        Type = 'StorageMetrics'
        AND log_group = 'prod1_autoprovision_88_100_fdbserver'
        AND TIME BETWEEN '2025-10-31 00:00:00'::timestamp_ntz
        AND '2025-11-06 00:00:00'::timestamp_ntz
        AND v:ClearRangeMutations IS NOT NULL
)
SELECT
    h.TIME as high_latency_time,
    h.MACHINE,
    h.latency_p99,
    SUM(r.op_get_range) as total_op_get_range_1min_before,
    SUM(r.op_nexts) as total_op_nexts_1min_before,
    SUM(s.clear_range_mutations) as total_clear_range_mutations_1min_before,
    SUM(s.ss_get_range) as total_ss_get_range_1min_before
FROM
    high_latency_events h
    LEFT JOIN redwood_metrics r ON h.MACHINE = r.MACHINE
        AND r.TIME BETWEEN DATEADD('minute', -1, h.TIME)
        AND DATEADD('second', 5, h.TIME)
    LEFT JOIN storage_metrics s ON h.MACHINE = s.MACHINE
        AND s.TIME BETWEEN DATEADD('minute', -1, h.TIME)
        AND DATEADD('second', 5, h.TIME)
GROUP BY
    h.TIME,
    h.MACHINE,
    h.latency_p99
-- HAVING
--     SUM(r.op_get_range) < 10000
--     AND SUM(r.op_nexts) < 10000
--     AND SUM(s.ss_get_range) > 100
ORDER BY
    h.TIME DESC;

In [ ]:
WITH high_latency_events AS (
    SELECT
        TIME,
        MACHINE,
        v:"P99"::float as latency_p99
    FROM
        SNOWHOUSE_IMPORT.PROD1.FDB_SERVER_LOGS_MAIN_V
    WHERE
        Type = 'NormalReadLatencyMetrics'
        AND v:"P99"::float > 0.05
        AND log_group = 'prod1fdb2_fdbserver'
        AND TIME BETWEEN '2025-10-31 00:00:00'::timestamp_ntz
        AND '2025-11-06 00:00:00'::timestamp_ntz
),
redwood_metrics AS (
    SELECT
        TIME,
        MACHINE,
        v:OpGetRange::float as op_get_range,
        COALESCE(v:OpTryNext::float, 0) + 
        COALESCE(v:OpNext::float, 0) + 
        COALESCE(v:OpNextBatch::float, 0) as op_nexts,
        v:OpNewIterator::float as op_it_new,
    FROM
        SNOWHOUSE_IMPORT.PROD1.FDB_SERVER_LOGS_MAIN_V
    WHERE
        Type = 'RedwoodMetrics'
        AND log_group = 'prod1fdb2_fdbserver'
        AND TIME BETWEEN '2025-10-31 00:00:00'::timestamp_ntz
        AND '2025-11-06 00:00:00'::timestamp_ntz
),
storage_metrics AS (
    SELECT
        TIME,
        MACHINE,
        split_part(v:ClearRangeMutations, ' ', 1)::float as clear_range_mutations,
        split_part(v:GetRangeQueries, ' ', 1)::float as ss_get_range
    FROM
        SNOWHOUSE_IMPORT.PROD1.FDB_SERVER_LOGS_MAIN_V
    WHERE
        Type = 'StorageMetrics'
        AND log_group = 'prod1fdb2_fdbserver'
        AND TIME BETWEEN '2025-10-31 00:00:00'::timestamp_ntz
        AND '2025-11-06 00:00:00'::timestamp_ntz
        AND v:ClearRangeMutations IS NOT NULL
)
SELECT
    h.TIME as high_latency_time,
    h.MACHINE,
    h.latency_p99,
    SUM(r.op_get_range) as total_op_get_range_1min_before,
    SUM(r.op_nexts) as total_op_nexts_1min_before,
    SUM(s.clear_range_mutations) as total_clear_range_mutations_1min_before,
    SUM(s.ss_get_range) as total_ss_get_range_1min_before,
    SUM(r.op_it_new) as total_op_it_news_1min_before
FROM
    high_latency_events h
    LEFT JOIN redwood_metrics r ON h.MACHINE = r.MACHINE
        AND r.TIME BETWEEN DATEADD('minute', -1, h.TIME)
        AND DATEADD('second', 5, h.TIME)
    LEFT JOIN storage_metrics s ON h.MACHINE = s.MACHINE
        AND s.TIME BETWEEN DATEADD('minute', -1, h.TIME)
        AND DATEADD('second', 5, h.TIME)
GROUP BY
    h.TIME,
    h.MACHINE,
    h.latency_p99
HAVING
    SUM(r.op_get_range) > 100000
    -- AND SUM(r.op_nexts) < 10000
    -- AND SUM(s.ss_get_range) > 100
ORDER BY
    h.TIME DESC;

In [ ]:
WITH slow_tasks AS (
    SELECT
        TIME,
        MACHINE,
        v:Duration::float as duration,
        v:NumYields as yield,
        v
    FROM
        SNOWHOUSE_IMPORT.PROD1.FDB_SERVER_LOGS_OLTP_V
    WHERE
        Type = 'SlowTask'
        AND v:Roles = 'SS'
        AND v:TaskID = '2500'
        AND log_group = 'prod1_autoprovision_88_100_fdbserver'
        AND TIME BETWEEN '2025-10-31 00:00:00'::timestamp_ntz
        AND '2025-11-06 00:00:00'::timestamp_ntz
),
latency_metrics AS (
    SELECT
        TIME,
        MACHINE,
        v:"P99"::float as latency_p99
    FROM
        SNOWHOUSE_IMPORT.PROD1.FDB_SERVER_LOGS_OLTP_V
    WHERE
        Type = 'NormalReadLatencyMetrics'
        AND log_group = 'prod1_autoprovision_88_100_fdbserver'
        AND TIME BETWEEN '2025-10-31 00:00:00'::timestamp_ntz
        AND '2025-11-06 00:00:00'::timestamp_ntz
)
SELECT
    st.TIME,
    st.MACHINE,
    st.duration,
    st.yield,
    st.v,
    MAX(lm.latency_p99) as max_latency_p99_1min_before
FROM
    slow_tasks st
    LEFT JOIN latency_metrics lm ON st.MACHINE = lm.MACHINE
        AND lm.TIME BETWEEN DATEADD('minute', -1, st.TIME)
        AND DATEADD('second', 5, st.TIME)
GROUP BY
    st.TIME,
    st.MACHINE,
    st.duration,
    st.yield,
    st.v
ORDER BY st.duration DESC;

In [ ]:
WITH high_latency_events AS (
    SELECT
        TIME,
        MACHINE,
        v:"P99"::float as latency_p99
    FROM
        SNOWHOUSE_IMPORT.PROD1.FDB_SERVER_LOGS_OLTP_V
    WHERE
        Type = 'NormalReadLatencyMetrics'
        AND v:"P99"::float > 0.5
        AND log_group = 'prod1_autoprovision_88_100_fdbserver'
        AND TIME BETWEEN '2025-10-31 00:00:00'::timestamp_ntz
        AND '2025-11-06 00:00:00'::timestamp_ntz
),
redwood_metrics AS (
    SELECT
        TIME,
        MACHINE,
        v:OpGetRange::float as op_get_range,
        COALESCE(v:OpTryNext::float, 0) + 
        COALESCE(v:OpNext::float, 0) + 
        COALESCE(v:OpNextBatch::float, 0) as op_nexts
    FROM
        SNOWHOUSE_IMPORT.PROD1.FDB_SERVER_LOGS_OLTP_V
    WHERE
        Type = 'RedwoodMetrics'
        AND log_group = 'prod1_autoprovision_88_100_fdbserver'
        AND TIME BETWEEN '2025-10-31 00:00:00'::timestamp_ntz
        AND '2025-11-06 00:00:00'::timestamp_ntz
),
storage_metrics AS (
    SELECT
        TIME,
        MACHINE,
        split_part(v:ClearRangeMutations, ' ', 1)::float as clear_range_mutations,
        split_part(v:GetRangeQueries, ' ', 1)::float as ss_get_range
    FROM
        SNOWHOUSE_IMPORT.PROD1.FDB_SERVER_LOGS_OLTP_V
    WHERE
        Type = 'StorageMetrics'
        AND log_group = 'prod1_autoprovision_88_100_fdbserver'
        AND TIME BETWEEN '2025-10-31 00:00:00'::timestamp_ntz
        AND '2025-11-06 00:00:00'::timestamp_ntz
        AND v:ClearRangeMutations IS NOT NULL
),
slow_task_metrics AS (
    SELECT
        TIME,
        MACHINE,
        v:Duration::float as duration
    FROM
        SNOWHOUSE_IMPORT.PROD1.FDB_SERVER_LOGS_OLTP_V
    WHERE
        Type = 'SlowTask'
        AND log_group = 'prod1_autoprovision_88_100_fdbserver'
        AND TIME BETWEEN '2025-10-31 00:00:00'::timestamp_ntz
        AND '2025-11-06 00:00:00'::timestamp_ntz
        AND v:Duration IS NOT NULL
)
SELECT
    h.TIME as high_latency_time,
    h.MACHINE,
    h.latency_p99,
    SUM(r.op_get_range) as total_op_get_range_1min_before,
    SUM(r.op_nexts) as total_op_nexts_1min_before,
    SUM(s.clear_range_mutations) as total_clear_range_mutations_1min_before,
    SUM(s.ss_get_range) as total_ss_get_range_1min_before,
    SUM(st.duration) as total_slow_task_duration_1min_before
FROM
    high_latency_events h
    LEFT JOIN redwood_metrics r ON h.MACHINE = r.MACHINE
        AND r.TIME BETWEEN DATEADD('minute', -1, h.TIME)
        AND DATEADD('second', 5, h.TIME)
    LEFT JOIN storage_metrics s ON h.MACHINE = s.MACHINE
        AND s.TIME BETWEEN DATEADD('minute', -1, h.TIME)
        AND DATEADD('second', 5, h.TIME)
    LEFT JOIN slow_task_metrics st ON h.MACHINE = st.MACHINE
        AND st.TIME BETWEEN DATEADD('minute', -1, h.TIME)
        AND DATEADD('second', 5, h.TIME)
GROUP BY
    h.TIME,
    h.MACHINE,
    h.latency_p99
-- HAVING
    -- SUM(r.op_get_range) < 100000
--     AND SUM(r.op_nexts) < 10000
--     AND SUM(s.ss_get_range) > 100
ORDER BY
    h.TIME DESC;

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import json

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Get data from slow_tasks SQL cell
df_slow_tasks = slow_tasks.to_pandas()

# Parse the JSON data in the V column to extract TaskID and Duration
task_data = []
for index, row in df_slow_tasks.iterrows():
    try:
        # Parse the JSON string in the V column
        v_data = json.loads(row['V'])
        task_id = v_data.get('TaskID', 'Unknown')
        duration = v_data.get('Duration', 0)
        
        task_data.append({
            'TaskID': task_id,
            'Duration': float(duration) if duration else 0
        })
    except (json.JSONDecodeError, TypeError, ValueError):
        # Skip rows with invalid JSON or missing data
        continue

# Convert to DataFrame
df_tasks = pd.DataFrame(task_data)

# Group by TaskID and sum Duration
task_summary = df_tasks.groupby('TaskID')['Duration'].sum().reset_index()

# Sort by Duration in descending order
task_summary = task_summary.sort_values('Duration', ascending=False)

# Display the results
print("Slow Tasks Summary - Grouped by TaskID, Ordered by Total Duration:")
print(task_summary.to_string(index=False))

# Also create Streamlit visualization
st.subheader("Slow Tasks Analysis")
st.write("**Tasks grouped by TaskID, ordered by total duration:**")
st.dataframe(task_summary)

# Create a bar chart
if not task_summary.empty:
    st.bar_chart(task_summary.set_index('TaskID')['Duration'])

In [ ]:
# Then, we can use the python name to turn cell2 into a Pandas dataframe
my_df = cell2.to_pandas()

# Chart the data
st.subheader("Chance of SNOW ❄️")
st.line_chart(my_df, x='SNOWDAY', y='CHANCE_OF_SNOW')

# Give it a go!
st.subheader("Try it out yourself and show off your skills 🥇")